In [ ]:
!mkdir /root/.kaggle
import json
kaggle = {"username":"yanapple","key":"53c4e2d6e293bb5024208c4fdab61b4e"}
with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump(kaggle, f)

!chmod 600 /root/.kaggle/kaggle.json
#Загружаем данные выбранного соревнования
!kaggle datasets download mhantor/russian-voice-dataset

! unzip russian-voice-dataset

100% 1.29G/1.30G [00:17<00:00, 55.9MB/s]
100% 1.30G/1.30G [00:17<00:00, 80.0MB/s]
Archive:  russian-voice-dataset.zip
  inflating: Disorder Voices/Disorder Voices/1.wav  
  inflating: Disorder Voices/Disorder Voices/10.wav  
  inflating: Disorder Voices/Disorder Voices/100.wav  
  inflating: Disorder Voices/Disorder Voices/1000.wav  
  inflating: Disorder Voices/Disorder Voices/1001.wav  
  inflating: Disorder Voices/Disorder Voices/1002.wav  
  inflating: Disorder Voices/Disorder Voices/1003.wav  
  inflating: Disorder Voices/Disorder Voices/1004.wav  
  inflating: Disorder Voices/Disorder Voices/1005.wav  
  inflating: Disorder Voices/Disorder Voices/1006.wav  
  inflating: Disorder Voices/Disorder Voices/1007.wav  
  inflating: Disorder Voices/Disorder Voices/1008.wav  
  inflating: Disorder Voices/Disorder Voices/1009.wav  
  inflating: Disorder Voices/Disorder Voices/101.wav  
  inflating: Disorder Voices/Disorder Voices/1010.wav  
  inflating: Disorder Voices/Disorder Voices/1011

Загрузка библиотек

In [ ]:
import os
import pandas as pd
import librosa
import numpy as np
from torch.utils.data import Dataset
import torch


Загрузка данных.

**main_dataset_dir** - общий путь к датасету. Для распаковки zip файла требуется явно указывать путь, без передач через переменные.

**text_path** - путь\название Excel файла, хранящего транскрипцию аудио.

**audio_path** - путь\название файла, хранящего wav файлы аудио.

Закомментировано до момента работоспособности

In [ ]:
audio_path = '/content/Disorder Voices/Disorder Voices'
text_path = '/content/Speeches.xlsx'
dir_name = '/content/Disorder Voices/Disorder Voices/'

In [ ]:
# main_dataset_dir = "/content/Disorder_Russian_Speech.zip"
# text_path = ""
# audio_path = ""

# !unzip /content/Disorder_Russian_Speech.zip

Archive:  /content/Disorder_Russian_Speech.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/Disorder_Russian_Speech.zip or
        /content/Disorder_Russian_Speech.zip.zip, and cannot find /content/Disorder_Russian_Speech.zip.ZIP, period.


In [ ]:
def data_load():
  file = pd.read_excel(text_path) #/Speeches.xlsx
  text_list = [sentence for sentence in file['Русская речь']][:2000]

  files_in_dir = os.listdir(audio_path)

  audio_path_list = []
  i = 1

  for e in range(1, 2001):
      file_name = f'{e}.wav'
      sampl = librosa.load(dir_name + file_name, sr=16000)[0]
      sampl = sampl[np.newaxis, :]
      audio_path_list.append(torch.Tensor(sampl))

  return audio_path_list, text_list

С помощью **char_map** функция **remove_characters** позволяет отфильтровать текстовые данные (данным образом из списка, полученного из Excel). Функция оставит лишь релевантные знаки и буквы, исключив ошибки вроде случайных, не несущих нам важности знаков: :№;@().

**char_map** - словарь для замены векторного значения буквы и обратно. Так же использовано для фильтра.

In [ ]:
char_map = {"а": 0, "б": 1, "в": 2, "г": 3, "д": 4, "е": 5, "ё": 6, "ж": 7, "з": 8, "и": 9, "й": 10,
            "к": 11, "л": 12, "м": 13, "н": 14, "о": 15, "п": 16, "р": 17, "с": 18, "т": 19, "у": 20,
            "ф": 21, "ч": 22, "ц": 23, "ш": 24, "щ": 25, "ъ": 26, "ы": 27, "ь": 28, "э": 29, "ю": 30,
            "я": 31, "х": 32, " ": 33, "[UNK]": 34, "[PAD]": 35}

def remove_characters(sentence):
  sentence = sentence.lower()
  sentence = ''.join(filter(lambda x: x in char_map, sentence))
  return sentence

**split_train_test** разделяет датасет на тренировочный и обучающие датасеты, требующиеся для тренировки модели и корректировки ее поведения с помощью обучающего датасета.
По дефолту разделяет датасет на train\test в пропорции 9 к 1.

In [ ]:
def split_train_test(X, y):
    from sklearn.model_selection import train_test_split

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
    return X_train, X_test, y_train, y_test

Класс **AudioDataset** принимает только класс **Dataset**.
Позволяет оформить датасет библиотеки Pytorch, который требуется для обучения модели.


**train_dataset** - датасет для обучения. Размера train_dataset.size()

**test_dataset** - датасет для тестирования. Размера test_dataset.size()

In [ ]:
class AudioDataset(Dataset):
    def __init__(self, audio_list, text_list):
        self.audio_list = audio_list
        self.text_list = text_list

    def __len__(self):
        return len(self.text_list)

    def __getitem__(self, index):
        audio = self.audio_list[index]
        text = self.text_list[index]
        return audio, text

X, y = data_load()
y = list(map(remove_characters, y))
X_train, X_test, y_train, y_test = split_train_test(X, y)
train_dataset = AudioDataset(X_train, y_train)
test_dataset = AudioDataset(X_test, y_test)